In [65]:
import sys
from pathlib import Path

# point this to your project root (where `src/` folder lives)
project_root = Path("/Users/ywxiu/jasp-multimodal-rag").resolve()
sys.path.append(str(project_root))

# now you can import
from src.ingestion.pdf_text_loader import (
    clean_page_header,
    convert_text_to_sections,
    reconstruct_equations,
    normalize_tables,
    split_markdown_by_sections,
    count_tokens
)


In [66]:
import importlib
from src.ingestion import pdf_text_loader

importlib.reload(pdf_text_loader)

from src.ingestion.pdf_text_loader import normalize_tables


In [67]:


data_dir = Path("/Users/ywxiu/jasp-multimodal-rag/data/raw")
pdf_name = "Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf"


In [68]:
docs = SimpleDirectoryReader(data_dir).load_data()
page99 = [d for d in docs if d.metadata.get("page_label") == "99"][0]
print("🔹Raw text (first 500 chars):")
print(page99.text)


🔹Raw text (first 500 chars):
   
95 | P a g e  
JASP 0.19.3 - Professor Mark Goss-Sampson 
Tukey – is one of the most commonly used tests and provides controlled Type I error for groups with 
the same sample size and equal group variance. 
Scheffe – controls for the overall confidence level when the group sample sizes are different. 
Sidak – is similar to Bonferroni but assumes that each comparison is independent of the others . 
Slightly more powerful than Bonferroni. 
 
EFFECT SIZE 
JASP provides 3 alternative effect size calculations for use with the independent group ANOVA tests: 
Eta squared (η 2) - accurate for the sample variance explained but overestimates the population 
variance. This can make it difficult to compare the effect of a single variable in different studies. 
Partial Eta squared (η p2) – this solves the problem relating to population variance overestimation , 
allowing for comparison of the effect of the same variable in different studies. 
Omega squared (ω2) – No

In [69]:
cleaned = clean_page_header(page99.text)
print("🔹After clean_page_header:")
print(cleaned[:500])


🔹After clean_page_header:
Tukey – is one of the most commonly used tests and provides controlled Type I error for groups with 
the same sample size and equal group variance. 
Scheffe – controls for the overall confidence level when the group sample sizes are different. 
Sidak – is similar to Bonferroni but assumes that each comparison is independent of the others . 
Slightly more powerful than Bonferroni. 
 
EFFECT SIZE 
JASP provides 3 alternative effect size calculations for use with the independent group ANOVA tests: 


In [70]:
sections = convert_text_to_sections(cleaned, "99")
print(f"🔹Found {len(sections)} sections on page 99")
for s in sections:
    print("----")
    print(f"Title: {s['title']}")
    print(s["content"][:400])


🔹Found 3 sections on page 99
----
Title: None
Tukey – is one of the most commonly used tests and provides controlled Type I error for groups with
the same sample size and equal group variance.
Scheffe – controls for the overall confidence level when the group sample sizes are different.
Sidak – is similar to Bonferroni but assumes that each comparison is independent of the others .
Slightly more powerful than Bonferroni.
----
Title: EFFECT SIZE (page 99)
JASP provides 3 alternative effect size calculations for use with the independent group ANOVA tests:
Eta squared (η 2) - accurate for the sample variance explained but overestimates the population
variance. This can make it difficult to compare the effect of a single variable in different studies.
Partial Eta squared (η p2) – this solves the problem relating to population variance overestimation ,
----
Title: RUNNING THE INDEPENDENT ANOVA (page 99)
Load Independent ANOVA diets.csv. This contains A column containing the 3 diets used (A,

In [71]:
for s in sections:
    s["content"] = reconstruct_equations(s["content"])
    s["content"] = normalize_tables(s["content"])

print("🔹After reconstruct_equations + normalize_tables:")
print(sections[1]["content"][:500])


🔹After reconstruct_equations + normalize_tables:
JASP provides 3 alternative effect size calculations for use with the independent group ANOVA tests:
Eta squared ( 2) - accurate for the sample variance explained but overestimates the population
variance. This can make it difficult to compare the effect of a single variable in different studies.
Partial Eta squared ( p2)  this solves the problem relating to population variance overestimation ,
allowing for comparison of the effect of the same variable in different studies.
Omega squared (2)  No


In [117]:
from llama_index.core import SimpleDirectoryReader

docs = SimpleDirectoryReader(
    input_dir="/Users/ywxiu/jasp-multimodal-rag/data/processed/markdown",
    recursive=False
).load_data()

print(len(docs), docs[0].metadata)
print(docs[0].text[:500])


1 {'file_path': '/Users/ywxiu/jasp-multimodal-rag/data/processed/markdown/Statistical-Analysis-in-JASP-A-guide-for-students-2025.md', 'file_name': 'Statistical-Analysis-in-JASP-A-guide-for-students-2025.md', 'file_type': 'text/markdown', 'file_size': 187828, 'creation_date': '2025-11-06', 'last_modified_date': '2025-11-06'}
# USING THE JASP ENVIRONMENT (page 6)


Open JASP.
The main menu can be accessed by clicking on the top-left icon.
Open:
JASP has its own . jasp format but can open a variety of
different dataset formats, such as:
.xls/xlsx (Excel files)
.csv (comma-separated values) can be saved in Excel
.txt (plain text) can also be saved in Excel
.tsv (tab-separated values) can also be saved in Excel
.sav (IBM SPSS data file)
.ods (Open Document spreadsheet)
.dta (Stata data file)
.por (SPSS ASCII file)
.Sas7


In [118]:
from llama_index.core.node_parser import SimpleNodeParser
import re

def split_markdown_by_sections(document_text):
    pattern = r"^# (.+?) \(page (\d+)\)"
    matches = list(re.finditer(pattern, document_text, flags=re.MULTILINE))
    sections = []

    for i, match in enumerate(matches):
        title, page = match.group(1), match.group(2)
        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(document_text)
        section_text = document_text[start:end].strip()

        sections.append({
            "text": section_text,
            "metadata": {
                "section_title": title,
                "page_start": page,
                "page_end": page,
            },
        })
    return sections


In [119]:
from llama_index.core import Document

documents = []
for doc in docs:
    for sec in split_markdown_by_sections(doc.text):
        documents.append(
            Document(
                text=sec["text"],
                metadata={
                    "pdf_name": doc.metadata.get("file_name"),
                    **sec["metadata"],
                }
            )
        )

print(f"✅ Created {len(documents)} section-level documents")


✅ Created 270 section-level documents


In [120]:
import tiktoken

# Choose the tokenizer for your target model
encoding = tiktoken.get_encoding("cl100k_base")  # for GPT-4-turbo / GPT-3.5-turbo

def count_tokens(text):
    return len(encoding.encode(text))

# Add token length as metadata
for doc in documents:
    n_tokens = count_tokens(doc.text)
    doc.metadata["token_length"] = n_tokens

print("✅ Token lengths added to metadata!")
print(documents[0].metadata)


✅ Token lengths added to metadata!
{'pdf_name': 'Statistical-Analysis-in-JASP-A-guide-for-students-2025.md', 'section_title': 'USING THE JASP ENVIRONMENT', 'page_start': '6', 'page_end': '6', 'token_length': 810}


In [123]:
import json, re, uuid, tiktoken
from datetime import datetime
from pathlib import Path
from typing import List
from llama_index.core import Document

def save_test_only_json(
    documents: List[Document],
    output_path: str,
    pdf_name: str,
    source_path: str,
):
    """
    Save section-level LlamaIndex Documents to a JSON file
    with minimal, ChromaDB-safe metadata.
    Handles case-insensitive placeholder extraction.
    
    Args:
        documents (List[Document]): List of LlamaIndex Document objects.
        output_path (str): Path to save the JSON file.
        pdf_name (str): Name of the original PDF source.
        source_path (str): Absolute path to the markdown file used.
    """

    encoding = tiktoken.get_encoding("cl100k_base")

    def count_tokens(text: str) -> int:
        return len(encoding.encode(text))

    def clean_none_values(d: dict) -> dict:
        """Remove keys with None or empty strings (Chroma-safe)."""
        return {k: v for k, v in d.items() if v not in [None, ""]}

    # --- Pattern: match image placeholders, case-insensitive ---
    placeholder_pattern = re.compile(r"\[image_placeholder_[a-z0-9_]+\]", re.IGNORECASE)

    records = []
    for doc in documents:
        text = doc.text
        meta = doc.metadata.copy()

        # --- Parse section title and page number ---
        section_title = meta.get("section_title", "")
        page_start = str(meta.get("page_start", ""))
        section_id = (
            re.sub(r"[^A-Za-z0-9]+", "_", section_title)
            .strip("_")
            .upper()
            + f"_PAGE_{page_start}"
        )

        # --- Extract image placeholders (case-insensitive) ---
        placeholders = placeholder_pattern.findall(text)
        # Normalize placeholders to uppercase for consistency
        placeholders = [p.upper() for p in placeholders]

        # --- Build metadata dict ---
        record_meta = {
            "doc_id": str(uuid.uuid4())[:8],
            "pdf_name": pdf_name,
            "source_path": source_path,
            "markdown_file": Path(source_path).name,
            "section_id": section_id,
            "section_title": section_title,
            "page_start": page_start,
            "image_placeholders": placeholders,
            "token_length": count_tokens(text),
            "processing_date": datetime.now().isoformat(timespec="seconds"),
        }

        # --- Remove null/empty keys ---
        record_meta = clean_none_values(record_meta)

        # --- Append record ---
        records.append({"text": text, "metadata": record_meta})

    # --- Save to JSON ---
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)

    with open(output_path, "w", encoding="utf-8") as f:
        json.dump(records, f, indent=2, ensure_ascii=False)

    print(f"✅ Saved {len(records)} section-level documents → {output_path}")
    print(f"📸 Example placeholders from second doc: {records[1]['metadata'].get('image_placeholders', [])}")


In [124]:
# Example call in your pipeline
save_test_only_json(
    documents=documents,
    output_path="/Users/ywxiu/jasp-multimodal-rag/data/processed/test_only.json",
    pdf_name="Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf",
    source_path="/Users/ywxiu/jasp-multimodal-rag/data/processed/markdown/Statistical-Analysis-in-JASP-A-guide-for-students-2025.md"
)


✅ Saved 270 section-level documents → /Users/ywxiu/jasp-multimodal-rag/data/processed/test_only.json
📸 Example placeholders from second doc: ['[IMAGE_PLACEHOLDER_USING_THE_JASP_ENVIRONMENT_PAGE_6_1]', '[IMAGE_PLACEHOLDER_USING_THE_JASP_ENVIRONMENT_PAGE_6_2]', '[IMAGE_PLACEHOLDER_USING_THE_JASP_ENVIRONMENT_PAGE_6_3]', '[IMAGE_PLACEHOLDER_USING_THE_JASP_ENVIRONMENT_PAGE_6_4]']


In [110]:
import json
from pathlib import Path

# Path for your output JSON file
output_path = Path("/Users/ywxiu/jasp-multimodal-rag/data/processed/text_only.json")

# Convert each Document to a plain dict
# `Document` objects have a .text and .metadata attribute
data_to_save = [
    {
        "text": doc.text,
        "metadata": doc.metadata
    }
    for doc in documents
]

# Write to disk
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(data_to_save, f, indent=2, ensure_ascii=False)

print(f"✅ Saved {len(documents)} section-level documents with token metadata → {output_path}")


✅ Saved 270 section-level documents with token metadata → /Users/ywxiu/jasp-multimodal-rag/data/processed/text_only.json


In [32]:
from llama_parse import LlamaParse

parser = LlamaParse(result_type="markdown")
docs = parser.load_data("/Users/ywxiu/jasp-multimodal-rag/data/test/test_pages25-28.pdf")

print(docs[0].text)


2025-11-05 18:13:49,076 - INFO - HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"


Started parsing the file under job_id 1226e0d5-f6b2-46c6-ac66-0837b6cc67b6


2025-11-05 18:13:50,295 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/1226e0d5-f6b2-46c6-ac66-0837b6cc67b6 "HTTP/1.1 200 OK"
2025-11-05 18:13:52,738 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/1226e0d5-f6b2-46c6-ac66-0837b6cc67b6 "HTTP/1.1 200 OK"
2025-11-05 18:13:56,043 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/1226e0d5-f6b2-46c6-ac66-0837b6cc67b6 "HTTP/1.1 200 OK"
2025-11-05 18:13:56,728 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/1226e0d5-f6b2-46c6-ac66-0837b6cc67b6/result/markdown "HTTP/1.1 200 OK"



# SPLITTING DATA FILES

If there is a grouping variable (categorical or ordinal), descriptive statistics and plots can be produced for each group. Using Descriptive data.csv with the variable data in the Variables box, now add Group to the Split box.

| Variables          | Descriptive Statistics |                         |       |         |
| ------------------ | ---------------------- | ----------------------- | ----- | ------- |
| Variable           |                        | Group 1                 |       | Group 2 |
|                    | Mean                   | 16.021                  |       | 18.787  |
| Std. Error of Mean |                        |                         | 0.362 | 0.316   |
|                    | Std. Deviation         |                         | 6.424 | 7.040   |
|                    | Skewness               |                         | 0.200 | -0.176  |
|                    | Std. Error of Skewness |                         | 0.137 | 0.110   |
|          

In [18]:
from llama_index.core import SimpleDirectoryReader, Document
from pathlib import Path

DATA_DIR = "/Users/ywxiu/jasp-multimodal-rag/data/test"
raw_docs = SimpleDirectoryReader(DATA_DIR).load_data()

enriched_docs = []
for doc in raw_docs:
    pdf_name = Path(doc.metadata["file_name"]).stem

    # Infer chapter and section from filename, if present
    parts = pdf_name.split("_")
    chapter = next((p for p in parts if p.lower().startswith("chapter")), None)
    section = next((p for p in parts if p.lower().startswith("section")), None)

    doc.metadata["chapter"] = chapter or "Unknown"
    doc.metadata["section"] = section or "Unknown"
    enriched_docs.append(doc)

print(enriched_docs[0].metadata)


{'page_label': '1', 'file_name': 'test_pages25-28.pdf', 'file_path': '/Users/ywxiu/jasp-multimodal-rag/data/test/test_pages25-28.pdf', 'file_type': 'application/pdf', 'file_size': 1716874, 'creation_date': '2025-10-24', 'last_modified_date': '2025-10-24', 'chapter': 'Unknown', 'section': 'Unknown'}


In [16]:
#Imports and configuration
import base64
import json
import requests
from pathlib import Path
from pdf2image import convert_from_path
from tqdm import tqdm
from pathlib import Path

# ---- CONFIG ----
PDF_PATH = Path("../data/test/test_pages25-28.pdf")      # path to your local PDF

OUTPUT_DIR = Path("data/test/image")     # directory to save page images
MODEL_NAME = "llava-phi3:latest"  # Ollama model name

OLLAMA_URL = "http://localhost:11434/api/generate"


In [17]:
PROMPT_TEMPLATE ="""

Summarize this image in a concise, academic tone relevant to a statistical software(JASP),such as interface screenshots or outputs.
"""

In [8]:
from llama_parse import LlamaParse
from llama_index.core import Document
from itertools import groupby
import fitz, base64, requests

def load_structured_manual(pdf_path):
    parser = LlamaParse(result_type="json")
    parsed = parser.load_data(pdf_path)   # returns pages with structured elements
    sections = []
    cur_section = None
    text_buffer = []

    for page in parsed:
        for el in page["elements"]:
            if el["type"] == "heading" and el["level"] in [1, 2]:
                if text_buffer:
                    sections.append((cur_section, "\n".join(text_buffer)))
                    text_buffer = []
                cur_section = el["text"]
            elif el["type"] == "paragraph":
                text_buffer.append(el["text"])
    if text_buffer:
        sections.append((cur_section, "\n".join(text_buffer)))

    docs = [
        Document(
            text=txt.strip(),
            metadata={
                "section": sec,
                "chapter": sec.split(":")[0] if ":" in sec else None,
                "source": pdf_path
            }
        )
        for sec, txt in sections if txt.strip()
    ]
    return docs


In [20]:
# LlamaParse is async-first, so we need to run this line of code if you are working on a notebook
import nest_asyncio

nest_asyncio.apply()


In [44]:
import os
os.environ["LLAMA_CLOUD_API_KEY"] = "llx-Hs5ASBbsMgrqi91UgmKJOIR0JdCMilbWWpaWMSTX7n7qcb7x"


In [41]:

import os
os.getenv("LLAMA_CLOUD_API_KEY")


''

In [45]:
import os
print(os.getcwd())


/Users/ywxiu/jasp-multimodal-rag


In [32]:
import os
os.chdir("/Users/ywxiu/jasp-multimodal-rag")
print(os.getcwd())  # should now show /Users/ywxiu/jasp-multimodal-rag


/Users/ywxiu/jasp-multimodal-rag


In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_parse import LlamaParse
documents = LlamaParse(result_type="text").load_data("./data/raw/Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf")


2025-10-23 18:18:02,708 - INFO - HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"


Started parsing the file under job_id cb7f9819-715f-4346-8408-620c826a8c67


2025-10-23 18:18:03,930 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:18:06,156 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:18:09,438 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:18:13,647 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:18:19,076 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:18:24,468 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:18:29,874 - INFO - HTTP Request: GET https://api.cloud.llamain

.

2025-10-23 18:18:51,656 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:18:57,094 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:19:02,507 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:19:07,944 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:19:13,388 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:19:18,833 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:19:24,267 - INFO - HTTP Request: GET https://api.cloud.llamain

.

2025-10-23 18:19:46,230 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:19:51,661 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:19:57,049 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:20:02,454 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:20:08,554 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:20:13,989 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:20:19,427 - INFO - HTTP Request: GET https://api.cloud.llamain

.

2025-10-23 18:20:41,137 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:20:46,589 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:20:52,000 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:20:57,423 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:21:02,855 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:21:08,342 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/cb7f9819-715f-4346-8408-620c826a8c67 "HTTP/1.1 200 OK"
2025-10-23 18:21:13,834 - INFO - HTTP Request: GET https://api.cloud.llamain

In [47]:
len(documents)

186

In [60]:
print(documents[91].text)


9  JASP

LOGISTIC REGRESSION
In simple and multiple linear regression outcome and predictor variable(s) were continuous data.
What if the outcome was a binary/categorical measure? Can, for example, a yes or no outcome be
predicted by other categorical or continuous variables? The answer is yes if binary logistic regression
is used. This method is used to predict the probability of a binary yes or no outcome.

The null hypothesis tested is that there is no relationship between the outcome and the predictor
variable(s).

As can be seen in the graph below, a linear regression line between the yes and no responses would
be meaningless as a prediction model. Instead, a sigmoidal logistic regression curve is fitted with a
minimum of 0 and a maximum of 1. Some predictor values overlap between yes and no. For example,
a prediction value of 5 would give an equal 50% probability of being a yes or no outcome. Thresholds
are therefore calculated to determine if a predictor data value will be clas

In [64]:
from llama_parse import LlamaParse

parser = LlamaParse(
    api_key="llx-Hs5ASBbsMgrqi91UgmKJOIR0JdCMilbWWpaWMSTX7n7qcb7x",
    result_type="text",
    config={
        "parsing_instruction": (
            "When you encounter plots in the document, "
            "insert a clear placeholder in the text like "
            "[FIGURE_PLACEHOLDER: page{page_number}_fig{index}]. "
            "Do not describe the image; just insert the marker in the correct position."
        )
    }
)

documents_new = parser.load_data("./data/raw/Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf")


/Users/ywxiu/Library/Caches/pypoetry/virtualenvs/jasp-multimodal-rag-F1fOabRm-py3.12/lib/python3.12/site-packages/llama_cloud_services/parse/base.py:551: UserWarning: The following parameters are unused: config.

 - 'config' is not a valid parameter. Please check the documentation or update the package.
  warnings.warn(
2025-10-23 19:19:18,368 - INFO - HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"


Started parsing the file under job_id 97c78384-e7b2-40b3-8225-6ac393bdd949


2025-10-23 19:19:19,591 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:19:21,764 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:19:24,971 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:19:29,164 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:19:34,599 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:19:40,006 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:19:45,464 - INFO - HTTP Request: GET https://api.cloud.llamain

.

2025-10-23 19:20:07,170 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:20:12,601 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:20:18,097 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:20:23,670 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:20:29,101 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:20:35,858 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:20:41,370 - INFO - HTTP Request: GET https://api.cloud.llamain

.

2025-10-23 19:21:05,016 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:21:10,713 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:21:16,447 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949 "HTTP/1.1 200 OK"
2025-10-23 19:21:16,858 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/97c78384-e7b2-40b3-8225-6ac393bdd949/result/text "HTTP/1.1 200 OK"


In [65]:
print(documents[91].text)


9  JASP

LOGISTIC REGRESSION
In simple and multiple linear regression outcome and predictor variable(s) were continuous data.
What if the outcome was a binary/categorical measure? Can, for example, a yes or no outcome be
predicted by other categorical or continuous variables? The answer is yes if binary logistic regression
is used. This method is used to predict the probability of a binary yes or no outcome.

The null hypothesis tested is that there is no relationship between the outcome and the predictor
variable(s).

As can be seen in the graph below, a linear regression line between the yes and no responses would
be meaningless as a prediction model. Instead, a sigmoidal logistic regression curve is fitted with a
minimum of 0 and a maximum of 1. Some predictor values overlap between yes and no. For example,
a prediction value of 5 would give an equal 50% probability of being a yes or no outcome. Thresholds
are therefore calculated to determine if a predictor data value will be clas

In [66]:
pip show llama-parse


Name: llama-parse
Version: 0.6.54
Summary: Parse files into RAG-Optimized formats.
Home-page: 
Author: 
Author-email: Logan Markewich <logan@llamaindex.ai>
License-Expression: MIT
Location: /Users/ywxiu/Library/Caches/pypoetry/virtualenvs/jasp-multimodal-rag-F1fOabRm-py3.12/lib/python3.12/site-packages
Requires: llama-cloud-services
Required-by: llama-index-readers-llama-parse
Note: you may need to restart the kernel to use updated packages.


In [72]:
from llama_parse import LlamaParse

parser = LlamaParse(
    api_key="llx-Hs5ASBbsMgrqi91UgmKJOIR0JdCMilbWWpaWMSTX7n7qcb7x",
    result_type="text",
    parse_mode="parse_document_with_llm"
)

documents_new = parser.load_data(
    "./data/raw/Statistical-Analysis-in-JASP-A-guide-for-students-2025.pdf",
    extra_info={
        "parsing_instruction": (
            "When you encounter plots in the document, "
            "insert a clear placeholder in the text like "
            "[FIGURE_PLACEHOLDER: page{page_number}_fig{index}]. "
            "Do not describe the image; just insert the marker in the correct position."
        )
    },
)


2025-10-23 19:37:56,058 - INFO - HTTP Request: POST https://api.cloud.llamaindex.ai/api/parsing/upload "HTTP/1.1 200 OK"


Started parsing the file under job_id fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04


2025-10-23 19:37:57,359 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:37:59,548 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:38:02,780 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:38:07,119 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:38:14,932 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:38:20,480 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:38:25,871 - INFO - HTTP Request: GET https://api.cloud.llamain

.

2025-10-23 19:38:47,854 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:38:53,264 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:38:58,715 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:39:07,410 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:39:12,853 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:39:18,272 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:39:23,713 - INFO - HTTP Request: GET https://api.cloud.llamain

.

2025-10-23 19:39:45,426 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:39:50,908 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:40:01,556 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:40:07,539 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:40:13,104 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:40:18,633 - INFO - HTTP Request: GET https://api.cloud.llamaindex.ai/api/parsing/job/fe1b5ba6-fad2-4a26-9496-2c91cc1bdb04 "HTTP/1.1 200 OK"
2025-10-23 19:40:24,033 - INFO - HTTP Request: GET https://api.cloud.llamain

In [75]:
len(documents_new)

1

In [76]:
print(documents_new[0].text)



    STATISTICAL ANALYSIS IN JASP

    A GUIDE FOR STUDENTS

                                JASP 2025

    Professor Mark A Goss-Sampson
    Professor Mark A Goss-Sampson


    7th Edition JASP v0.19.3 2025

    Copyright © 2025 by Mark A Goss-Sampson.

    Licenced as CC BY 4.0

    All rights reserved. This book or any portion thereof may not be reproduced or used in any manner
    whatsoever without the express written permission of the author except for research, education or
    private study.

                                                   9  JASP

    CONTENTS

    PREFACE                                             1
    USING THE JASP ENVIRONMENT                          2
    DATA HANDLING IN JASP                               8
    DATA EDITING IN JASP                               11
    JASP ANALYSIS MENU                                 12
    DESCRIPTIVE STATISTICS                             15
     SPLITTING DATA FILES                              21
    DESCRIPTI

In [77]:
print(parser.parse_mode)


parse_document_with_llm
